# Mapping ATC RxNORM

In [1]:
# Import des librairie
import pandas as pd

In [2]:
# Tables de vocabulaire OMOP

# Table concept
concept = pd.read_csv('concept.csv', sep='\t')
concept.head()

C:\Users\GaëlleRimaud\AppData\Local\Temp\ipykernel_30096\3291131607.py:4: DtypeWarning: Columns (5,6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  concept = pd.read_csv('concept.csv', sep='\t')


,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,45756805,Pediatric Cardiology,Provider,ABMS,Physician Specialty,S,OMOP4821938,19700101,20991231,NaN
1,45756804,Pediatric Anesthesiology,Provider,ABMS,Physician Specialty,S,OMOP4821939,19700101,20991231,NaN
2,45756803,Pathology-Anatomic / Pathology-Clinical,Provider,ABMS,Physician Specialty,S,OMOP4821940,19700101,20991231,NaN
3,45756802,Pathology - Pediatric,Provider,ABMS,Physician Specialty,S,OMOP4821941,19700101,20991231,NaN
4,45756801,Pathology - Molecular Genetic,Provider,ABMS,Physician Specialty,S,OMOP4821942,19700101,20991231,NaN


In [3]:
# Table concept_relationship --> Table qui contient les relations entre les codes (alignements, hiérarchiques, etc)
concept_rel = pd.read_csv('CONCEPT_RELATIONSHIP.csv', sep='\t')
concept_rel.head()

,concept_id_1,concept_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason
0,42628856,42628856,Mapped from,20160606,20991231,NaN
1,42628856,42628856,Maps to,20160606,20991231,NaN
2,42628859,40165064,Has quantified form,20160606,20991231,NaN
3,42628859,42628859,Mapped from,20160606,20991231,NaN
4,42628859,42628859,Maps to,20160606,20991231,NaN


Table concept pour les codes ATC

In [27]:
def select_concept_for_vocab(concept,vocab, info=True):
    specialized_concept = concept[concept.vocabulary_id==vocab][['concept_code','concept_id','concept_name','concept_class_id']]
    specialized_concept = specialized_concept.rename(columns={'concept_code' : f'{vocab}_code', 
        'concept_id': f'{vocab}_concept_id',
        'concept_name':f'{vocab}_libelle',
        'concept_class_id' : f'{vocab}_concept_class_id'})

    # Nombre de valeurs uniques par colonne
    print('Nombre de valeurs uniques par colonne\n', specialized_concept.nunique())

    # Nombre de lignes par classe
    print('\nNombre de lignes par classe\n', specialized_concept[f'{vocab}_concept_class_id'].value_counts())

    return specialized_concept

In [28]:
atc = select_concept_for_vocab(concept, 'ATC')
atc.head()

Nombre de valeurs uniques par colonne
 ATC_code                6740
ATC_concept_id          6740
ATC_libelle             6621
ATC_concept_class_id       5
dtype: int64

Nombre de lignes par classe
 ATC 5th    5452
ATC 4th     911
ATC 3rd     269
ATC 2nd      94
ATC 1st      14
Name: ATC_concept_class_id, dtype: int64


,ATC_code,ATC_concept_id,ATC_libelle,ATC_concept_class_id
175,A,21600001,ALIMENTARY TRACT AND METABOLISM,ATC 1st
176,A01,21600002,STOMATOLOGICAL PREPARATIONS,ATC 2nd
177,A01A,21600003,STOMATOLOGICAL PREPARATIONS,ATC 3rd
178,A01AA,21600004,Caries prophylactic agents,ATC 4th
179,A01AA01,21600005,"sodium fluoride; oral, local oral (caries prop...",ATC 5th


Table concept pour les codes RxNorm

In [29]:
rxnorm = select_concept_for_vocab(concept,'RxNorm')
rxnorm.head()

Nombre de valeurs uniques par colonne
 RxNorm_code                308709
RxNorm_concept_id          308709
RxNorm_libelle             306364
RxNorm_concept_class_id        18
dtype: int64

Nombre de lignes par classe
 Clinical Drug           53083
Branded Drug            39083
Clinical Drug Comp      38474
Branded Drug Comp       36748
Branded Drug Form       26595
Branded Dose Group      23388
Clinical Drug Form      20773
Brand Name              19469
Clinical Dose Group     17741
Ingredient              15604
Multiple Ingredients     4469
Quant Clinical Drug      3860
Precise Ingredient       3476
Quant Branded Drug       3215
Branded Pack             1308
Clinical Pack            1175
Dose Form                 201
Dose Form Group            47
Name: RxNorm_concept_class_id, dtype: int64


,RxNorm_code,RxNorm_concept_id,RxNorm_libelle,RxNorm_concept_class_id
63745,1042838,40229458,benzocaine 0.18 MG/MG Oral Gel,Clinical Drug
63746,1042847,40229459,benzocaine 0.18 MG/MG Oral Ointment,Clinical Drug
63747,1049224,40232055,benzocaine 0.18 MG/MG Oral Ointment [Topicale],Branded Drug
63748,1371295,43525657,benzocaine 180 MG/ML Mucous Membrane Topical S...,Clinical Drug
63749,1371298,43525658,benzocaine 180 MG/ML Mucous Membrane Topical S...,Branded Drug


Codes ATC dans la table des relations

In [32]:
def merge_concept_to_concept_rel(specialized_concept, concept_rel, vocab):
    join_ = specialized_concept.merge(concept_rel, left_on=[f'{vocab}_concept_id'], right_on=['concept_id_1'])

    # Valeurs uniques par colonne
    print('Nombre de valeurs uniques par colonne\n', join_.nunique())
    return join_

In [33]:
atc_concept_rel = merge_concept_to_concept_rel(atc, concept_rel, 'ATC')
atc_concept_rel.head()

Nombre de valeurs uniques par colonne
 ATC_code                 6735
ATC_concept_id           6735
ATC_libelle              6616
ATC_concept_class_id        5
concept_id_1             6735
concept_id_2            26098
relationship_id            11
valid_start_date           46
valid_end_date              1
invalid_reason              0
dtype: int64


,ATC_code,ATC_concept_id,ATC_libelle,ATC_concept_class_id,concept_id_1,concept_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason
0,A,21600001,ALIMENTARY TRACT AND METABOLISM,ATC 1st,21600001,21600490,Subsumes,20120815,20991231,NaN
1,A,21600001,ALIMENTARY TRACT AND METABOLISM,ATC 1st,21600001,21600509,Subsumes,20120815,20991231,NaN
2,A,21600001,ALIMENTARY TRACT AND METABOLISM,ATC 1st,21600001,21600531,Subsumes,20120815,20991231,NaN
3,A,21600001,ALIMENTARY TRACT AND METABOLISM,ATC 1st,21600001,21600600,Subsumes,20120815,20991231,NaN
4,A,21600001,ALIMENTARY TRACT AND METABOLISM,ATC 1st,21600001,21600697,Subsumes,20120815,20991231,NaN


Relations entre les codes ATC et RxNorm

In [35]:
atc_rxnorm = atc_concept_rel.merge(rxnorm, left_on=['concept_id_2'], right_on=['RxNorm_concept_id']).drop(columns=['valid_start_date','valid_end_date','invalid_reason','concept_id_1','concept_id_2'])
atc_rxnorm.head()

,ATC_code,ATC_concept_id,ATC_libelle,ATC_concept_class_id,relationship_id,RxNorm_code,RxNorm_concept_id,RxNorm_libelle,RxNorm_concept_class_id
0,A01AA01,21600005,"sodium fluoride; oral, local oral (caries prop...",ATC 5th,ATC - RxNorm pr lat,9873,19069022,sodium fluoride,Ingredient
1,A01AA01,21600005,"sodium fluoride; oral, local oral (caries prop...",ATC 5th,Maps to,9873,19069022,sodium fluoride,Ingredient
2,A01AA30,21600009,"combinations; oral, local oral (caries prophyl...",ATC 5th,ATC - RxNorm pr up,9873,19069022,sodium fluoride,Ingredient
3,A01AA51,21600010,"sodium fluoride, combinations; oral, local oral",ATC 5th,ATC - RxNorm pr lat,9873,19069022,sodium fluoride,Ingredient
4,A01AA51,21600010,"sodium fluoride, combinations; oral, local oral",ATC 5th,Maps to,9873,19069022,sodium fluoride,Ingredient


In [36]:
# Nombre de valeurs uniques par colonnes
atc_rxnorm.nunique()

ATC_code                    4703
ATC_concept_id              4703
ATC_libelle                 4698
ATC_concept_class_id           2
relationship_id                7
RxNorm_code                17588
RxNorm_concept_id          17588
RxNorm_libelle             17580
RxNorm_concept_class_id        8
dtype: int64

In [38]:
# Nombre de valeurs uniques par classe ATC
atc_rxnorm.ATC_concept_class_id.value_counts()

ATC 5th    81695
ATC 4th        8
Name: ATC_concept_class_id, dtype: int64

In [39]:
# Nombre de valeurs uniques par type de relation
atc_rxnorm.relationship_id.value_counts()

ATC - RxNorm sec up     36571
ATC - RxNorm            30084
ATC - RxNorm pr lat      5078
Maps to                  5062
ATC - RxNorm pr up       3622
ATC - RxNorm sec lat     1278
Subsumes                    8
Name: relationship_id, dtype: int64

In [40]:
atc_rxnorm.to_csv('mapping_athena_atc_rxnorm.csv', index=False)